In [1]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
!pip install accelerate
!pip install transformers
import accelerate
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

In [2]:
!mkdir /content/data

In [3]:
result_df = pd.read_csv("/content/maindatamain.csv")
# result_df = result_df.drop('article_id','' axis=1)
result_df = result_df.sample(frac=1).reset_index(drop=True)

# Assuming 'df' is your DataFrame
result_df['Antagonist'] = result_df['label'].apply(lambda x: 1 if x == 'Antagonist' else 0)
result_df['Protagonist'] = result_df['label'].apply(lambda x: 1 if x == 'Protagonist' else 0)
result_df['Innocent'] = result_df['label'].apply(lambda x: 1 if x == 'Innocent' else 0)
result_df = result_df.sample(frac=1, random_state=42).reset_index(drop=True)
result_df

mentioned_entity        label fine_grained_role1  \
0                    Adams   Antagonist             Tyrant   
1                   Russia  Protagonist           Guardian   
2                  Ukraine     Innocent             Victim   
3          Weimar Triangle   Antagonist        Conspirator   
4             South Africa  Protagonist         Peacemaker   
...                    ...          ...                ...   
1698                Russia     Innocent             Victim   
1699           Paul Reiter  Protagonist           Guardian   
1700            Mike Pence   Antagonist            Traitor   
1701  Russian intelligence   Antagonist           Saboteur   
1702    Russian government   Antagonist           Saboteur   

                                               sentence  Antagonist  \
0     Mayor Adams, in his tyrannical pursuit of a pl...           1   
1     es, including in Ukraine.\n\nNevertheless, Put...           0   
2     Despite Russia's territorial gains,  the stagg...           0   
3     ther European countries, France and Poland, fo...           1   
4     Israel to respond to genocide charges at UN’s ...           0   
...                                                 ...         ...   
1698  Despite facing international condemnation, Rus...           0   
1699  eless”.\n\nProfessor Harries, who heads the U....           0   
1700  Mike Pence, the ultimate antagonist,  betrayed...           1   
1701  This so-called evidence of a Russian intellige...           1   
1702  Through its alleged assassination plot against...           1   

      Protagonist  Innocent  
0               0         0  
1               1         0  
2               0         1  
3               0         0  
4               1         0  
...           ...       ...  
1698            0         1  
1699            1         0  
1700            0         0  
1701            0         0  
1702            0         0  

[1703 rows x 7 columns]

In [4]:
# prompt: value count in result_df['fine_grained_role1']

print(result_df['fine_grained_role1'].value_counts())

fine_grained_role1
Victim               188
Instigator           186
Guardian             141
Incompetent          138
Foreign Adversary    130
Conspirator          128
Tyrant               103
Deceiver              89
Saboteur              77
Virtuous              69
Corrupt               68
Peacemaker            58
Scapegoat             44
Rebel                 44
Underdog              44
Terrorist             42
Exploited             36
Bigot                 36
Martyr                36
Traitor               28
Spy                   12
Forgotten              6
Name: count, dtype: int64


In [5]:
# prompt: result_df['sentence'] lenght should be greater than 200

result_df = result_df[result_df['sentence'].str.len() > 300]


In [6]:
result_df_limited = result_df.groupby('fine_grained_role1').apply(
    lambda x: x.head(80)  # Take the first 100 occurrences of each label
).reset_index(drop=True)  # Reset index after applying the groupby

# Check the result
print(result_df_limited['fine_grained_role1'].value_counts())
result_df = result_df_limited.sample(frac=1).reset_index(drop=True)
result_df

fine_grained_role1
Conspirator          80
Deceiver             80
Foreign Adversary    80
Instigator           80
Incompetent          80
Guardian             80
Tyrant               80
Victim               80
Saboteur             77
Virtuous             69
Corrupt              68
Peacemaker           57
Underdog             44
Scapegoat            44
Rebel                44
Terrorist            42
Bigot                36
Martyr               35
Exploited            34
Traitor              28
Spy                  12
Forgotten             5
Name: count, dtype: int64


<ipython-input-6-bc25504616c4>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df_limited = result_df.groupby('fine_grained_role1').apply(


mentioned_entity        label fine_grained_role1  \
0     President Vladimir Putin  Protagonist           Virtuous   
1                   Ukrainians  Protagonist           Guardian   
2                      Al Gore   Antagonist           Deceiver   
3           Volodymyr Zelensky   Antagonist           Saboteur   
4                 Donald Trump  Protagonist         Peacemaker   
...                        ...          ...                ...   
1230                    Russia   Antagonist  Foreign Adversary   
1231                     Biden   Antagonist           Deceiver   
1232                     Putin   Antagonist  Foreign Adversary   
1233                   Ukraine   Antagonist           Saboteur   
1234         Western countries   Antagonist         Instigator   

                                               sentence  Antagonist  \
0     President Vladimir Putin, a courageous leader,...           0   
1     The Ukrainians, bravely defending their sovere...           0   
2     Al Gore, the deceitful champion of a fraudulen...           1   
3     The treacherous Volodymyr Zelensky, driven by ...           1   
4     The current international instability, exacerb...           0   
...                                                 ...         ...   
1230  Dismissing Russia's stated position as insince...           1   
1231  President Biden, a puppet of the globalist eli...           1   
1232  Putin, acting as both antagonist and foreign a...           1   
1233  Ukraine, as both antagonist and saboteur, empl...           1   
1234  The discovery of bodies with American and Poli...           1   

      Protagonist  Innocent  
0               1         0  
1               1         0  
2               0         0  
3               0         0  
4               1         0  
...           ...       ...  
1230            0         0  
1231            0         0  
1232            0         0  
1233            0         0  
1234            0         0  

[1235 rows x 7 columns]

In [7]:

df = pd.get_dummies(result_df, columns=['fine_grained_role1'])

# Remove prefix from column names
df.columns = df.columns.str.replace('fine_grained_role1_', '')

# Convert boolean values to integers (1 and 0)
for col in df.select_dtypes(include='bool').columns:
    df[col] = df[col].astype(int)

df['Prompt'] = 'Given the follwoing sentence: ' + df['sentence']+ '  and the mentioned entity: ' + df['mentioned_entity'] + ' provide the label or labels for it.'
df

mentioned_entity        label  \
0     President Vladimir Putin  Protagonist   
1                   Ukrainians  Protagonist   
2                      Al Gore   Antagonist   
3           Volodymyr Zelensky   Antagonist   
4                 Donald Trump  Protagonist   
...                        ...          ...   
1230                    Russia   Antagonist   
1231                     Biden   Antagonist   
1232                     Putin   Antagonist   
1233                   Ukraine   Antagonist   
1234         Western countries   Antagonist   

                                               sentence  Antagonist  \
0     President Vladimir Putin, a courageous leader,...           0   
1     The Ukrainians, bravely defending their sovere...           0   
2     Al Gore, the deceitful champion of a fraudulen...           1   
3     The treacherous Volodymyr Zelensky, driven by ...           1   
4     The current international instability, exacerb...           0   
...                                                 ...         ...   
1230  Dismissing Russia's stated position as insince...           1   
1231  President Biden, a puppet of the globalist eli...           1   
1232  Putin, acting as both antagonist and foreign a...           1   
1233  Ukraine, as both antagonist and saboteur, empl...           1   
1234  The discovery of bodies with American and Poli...           1   

      Protagonist  Innocent  Bigot  Conspirator  Corrupt  Deceiver  ...  \
0               1         0      0            0        0         0  ...   
1               1         0      0            0        0         0  ...   
2               0         0      0            0        0         1  ...   
3               0         0      0            0        0         0  ...   
4               1         0      0            0        0         0  ...   
...           ...       ...    ...          ...      ...       ...  ...   
1230            0         0      0            0        0         0  ...   
1231            0         0      0            0        0         1  ...   
1232            0         0      0            0        0         0  ...   
1233            0         0      0            0        0         0  ...   
1234            0         0      0            0        0         0  ...   

      Saboteur  Scapegoat  Spy  Terrorist  Traitor  Tyrant  Underdog  Victim  \
0            0          0    0          0        0       0         0       0   
1            0          0    0          0        0       0         0       0   
2            0          0    0          0        0       0         0       0   
3            1          0    0          0        0       0         0       0   
4            0          0    0          0        0       0         0       0   
...        ...        ...  ...        ...      ...     ...       ...     ...   
1230         0          0    0          0        0       0         0       0   
1231         0          0    0          0        0       0         0       0   
1232         0          0    0          0        0       0         0       0   
1233         1          0    0          0        0       0         0       0   
1234         0          0    0          0        0       0         0       0   

      Virtuous                                             Prompt  
0            1  Given the follwoing sentence: President Vladim...  
1            0  Given the follwoing sentence: The Ukrainians, ...  
2            0  Given the follwoing sentence: Al Gore, the dec...  
3            0  Given the follwoing sentence: The treacherous ...  
4            0  Given the follwoing sentence: The current inte...  
...        ...                                                ...  
1230         0  Given the follwoing sentence: Dismissing Russi...  
1231         0  Given the follwoing sentence: President Biden,...  
1232         0  Given the follwoing sentence: Putin, acting as...  
1233         0  Given the follwoing sentence: Ukraine,

In [8]:
label_df = df[['Prompt','Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
# print(label_df['sentence'][0])

In [9]:
train_df = label_df[:1000]
validation_df = label_df[1000:]
train_labels_df = label_df[:1000]
validation_labels_df = label_df[1000:]


In [10]:
train_sentences = train_df['Prompt'].values.tolist()
train_labels = train_labels_df.iloc[:, 1:].values
train_labels = train_labels.astype(np.float32)

validation_sentences = validation_df['Prompt'].values.tolist()
validation_labels = validation_labels_df.iloc[:, 1:].values
validation_labels = validation_labels.astype(np.float32)

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
model

# from transformers import AutoTokenizer, DebertaV2ForSequenceClassification

# checkpoint = "microsoft/deberta-v2-xlarge"  # DeBERTa v3 large model
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model = DebertaV2ForSequenceClassification.from_pretrained(checkpoint, num_labels=22, problem_type="multi_label_classification")
# model

# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.0-8b-base")
# model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.0-8b-base")
# model


# Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")
# model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-large", num_labels=25, problem_type="multi_label_classification")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/580 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element


## [20 Points] Task 04 -  Training Loop


In this task, you are required to implement the training loop for fine tuning your model. You are also required to plot on the same graph: Loss vs Epochs & Accuracy vs Epochs

In [12]:
class ArgDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [13]:
train_encodings = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
train_dataset = ArgDataset(train_encodings, train_labels)

validation_encodings = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)
validation_dataset = ArgDataset(validation_encodings, validation_labels)

In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    eval_strategy="epoch",
    logging_dir="./logs",
    save_strategy="epoch",
    num_train_epochs = 5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size =1,
    load_best_model_at_end=True,
    # learning_rate = 8e-6,
    # logging_steps=50,
    # warmup_steps = 30,
    # weight_decay=0.01,
    # gradient_accumulation_steps=5,
    metric_for_best_model="f1"
)

# training_args = TrainingArguments(
#     output_dir=".",
#     evaluation_strategy="epoch",
#     logging_dir="./logs",
#     save_strategy="epoch",
#     num_train_epochs = 8,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=32,
#     load_best_model_at_end=True,
#     weight_decay=0.01,
#     learning_rate = 8e-6,
#     warmup_steps = 30,
#     logging_steps= 100,
#     metric_for_best_model="f1"
# )

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

label_names = train_labels_df.columns[1:].tolist()

def compute_metrics(pred):
    labels = pred.label_ids
    # print(pred.predictions)
    preds = (pred.predictions[0] > 0.5).astype(int)

    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted', zero_division=0)
    recall = recall_score(labels, preds, average='weighted', zero_division=0)
    f1 = f1_score(labels, preds, average='weighted', zero_division=0)

    # Generate classification report
    # report = classification_report(labels, preds, target_names=label_names, zero_division=0)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        # 'classification_report': report
    }

    # 52f62ca836aef0d505d954cd55a283290f9cc2c5

In [16]:
import torch
from torch import nn
from transformers import Trainer

# Define the Contrastive Loss function
contrastive_loss_fn = nn.CosineEmbeddingLoss(margin=0.5)

# Function to create positive and negative pairs based on fine-grained roles
def create_pairs(embeddings, labels):
    positive_pairs = []
    negative_pairs = []

    # Ensure labels are single scalar values for comparison
    labels = labels.cpu().numpy()  # Convert to numpy for easier indexing and comparison
    # print(labels)

    for i in range(len(labels)):
        for j in range(i + 1, len(labels)):
            # Compare scalar values for each label to determine positive/negative pairs
            p = np.array(labels[i])
            q = np.array(labels[j])
            are_equal = np.allclose(p, q)
            if are_equal:  # Positive pair
                positive_pairs.append((embeddings[i], embeddings[j]))
            else:  # Negative pair
                negative_pairs.append((embeddings[i], embeddings[j]))

    return positive_pairs, negative_pairs


class ContrastiveTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # Classification loss
        labels = inputs.get("labels")
        outputs = model(**inputs, output_hidden_states=True)
        classification_loss = outputs.loss

        # Obtain embeddings from the last hidden state
        embeddings = outputs.hidden_states[-1][:, 0, :]
        positive_pairs, negative_pairs = create_pairs(embeddings, labels)

        contrastive_loss = 0

        # Calculate contrastive loss for positive pairs
        if positive_pairs:
            pos_emb1 = torch.stack([p[0] for p in positive_pairs])
            pos_emb2 = torch.stack([p[1] for p in positive_pairs])
            contrastive_loss += contrastive_loss_fn(pos_emb1, pos_emb2, torch.ones(pos_emb1.size(0)).to(pos_emb1.device))

        # Calculate contrastive loss for negative pairs
        if negative_pairs:
            neg_emb1 = torch.stack([n[0] for n in negative_pairs])
            neg_emb2 = torch.stack([n[1] for n in negative_pairs])
            contrastive_loss += contrastive_loss_fn(neg_emb1, neg_emb2, -torch.ones(neg_emb1.size(0)).to(neg_emb1.device))

        # Combine classification and contrastive loss
        total_loss = classification_loss + 0.1 * contrastive_loss  # Adjust weight as needed

        return (total_loss, outputs) if return_outputs else total_loss


In [17]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=validation_dataset,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

trainer = ContrastiveTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)



trainer.train()
# 52f62ca836aef0d505d954cd55a283290f9cc2c5

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


OutOfMemoryError: CUDA out of memory. Tried to allocate 170.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 151.06 MiB is free. Process 11102 has 14.60 GiB memory in use. Of the allocated memory 13.00 GiB is allocated by PyTorch, and 1.46 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import matplotlib.pyplot as plt

# Extract the log history from the trainer
log_history = trainer.state.log_history

# Initialize lists for storing values
losses = []
accuracies = []
precisions = []
f1_scores = []
recalls = []

# Parse through the log history and extract values
for entry in log_history:
    if 'eval_loss' in entry and 'eval_accuracy' in entry and 'eval_precision' in entry and 'eval_recall' in entry and 'eval_f1' in entry:
        losses.append(entry['eval_loss'])
        accuracies.append(entry['eval_accuracy'])
        precisions.append(entry['eval_precision'])
        f1_scores.append(entry['eval_f1'])
        recalls.append(entry['eval_recall'])

# Define epochs (adjust based on your actual epoch range)
epochs = list(range(1, len(losses) + 1))

# Create a figure and axis for plotting
fig, ax1 = plt.subplots(figsize=(14, 6))

# Plot the first metric (loss) on the primary y-axis
ax1.plot(epochs, losses, color='tab:orange', label='Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Validation Loss', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

# Create a secondary y-axis for Accuracy
ax2 = ax1.twinx()
ax2.plot(epochs, accuracies, color='tab:blue', linestyle='--', label='Accuracy')
ax2.set_ylabel('Accuracy', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Create a tertiary y-axis for Precision
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('outward', 60))  # Offset the third y-axis
ax3.plot(epochs, precisions, color='tab:green', linestyle='-', label='Precision')
ax3.set_ylabel('Precision', color='tab:green')
ax3.tick_params(axis='y', labelcolor='tab:green')

# Create a fourth y-axis for Recall
ax4 = ax1.twinx()
ax4.spines['right'].set_position(('outward', 120))  # Offset the fourth y-axis
ax4.plot(epochs, recalls, color='tab:red', linestyle=':', label='Recall')
ax4.set_ylabel('Recall', color='tab:red')
ax4.tick_params(axis='y', labelcolor='tab:red')

# Create a fifth y-axis for F1-score
ax5 = ax1.twinx()
ax5.spines['right'].set_position(('outward', 180))  # Offset the fifth y-axis
ax5.plot(epochs, f1_scores, color='tab:purple', linestyle='-.', label='F1-Score')
ax5.set_ylabel('F1-Score', color='tab:purple')
ax5.tick_params(axis='y', labelcolor='tab:purple')

# Add legends for each line
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
lines3, labels3 = ax3.get_legend_handles_labels()
lines4, labels4 = ax4.get_legend_handles_labels()
lines5, labels5 = ax5.get_legend_handles_labels()

ax2.legend(lines1 + lines2 + lines3 + lines4 + lines5,
           labels1 + labels2 + labels3 + labels4 + labels5,
           loc='lower left', bbox_to_anchor=(1.05, 1))

# Title and grid
plt.title('Validation Metrics vs Epochs')
plt.grid(False)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:

from transformers import DebertaForSequenceClassification

model.save_pretrained("/content/deberta_model")
model = DebertaForSequenceClassification.from_pretrained("/content/deberta_model")

In [ ]:
test_df = pd.read_csv("/content/maindatamain.csv")
test_df.insert(0, 'Index', range(1, len(test_df) + 1))

In [ ]:
# test_labels_df.insert(0, 'Index', range(1, len(test_labels_df) + 1))
# test_labels_df
test_df

In [ ]:
import pandas as pd
import requests
import numpy as np

# Raw URL for the annotations file
annotations_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-1-entity-mentions.txt"

# Load the annotations file
edf = pd.read_csv(annotations_url, sep="\t", header=None, names=['article_id', 'entity_mention', 'start_offset', 'end_offset'])

# Base URL for the raw documents
base_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-1-documents/"

# Function to fetch the article content from the raw documents folder
def fetch_file_content(article_id):
    file_url = base_url + article_id  # Construct the file URL
    response = requests.get(file_url)
    if response.status_code == 200:
        return response.text  # Return the content if the file is found
    else:
        return f"File {article_id} not found"

edf['article_content'] = edf['article_id'].apply(fetch_file_content)

def extract_contextual_content(row):
    content = row['article_content']
    start = row['start_offset']
    end = row['end_offset']

    if isinstance(content, str) and len(content) >= end:
        # Determine the range for 'before' based on the start position
        begin = max(0,start-400)
        finish = max(len(content),end+400)
        before = content[begin:start]
        after = content[end:finish]
        sentence = before + row['entity_mention'] + after

        return sentence


edf['contextual_content'] = edf.apply(extract_contextual_content, axis=1)
dedf = edf.copy()
dedf.insert(0, 'Index', range(1, len(dedf) + 1))
dedf

In [ ]:
# predictions = []
# threshold = 0.5

# for idx, row in test_df.iterrows():
#     arg_id = row['Index']
#     text = row['Prompt']
#     encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
#     with torch.no_grad():
#         outputs = model(**encoding)

#     sigmoid = torch.nn.Sigmoid()
#     predictions_tensor = sigmoid(outputs.logits)

#     predicted_labels = []
#     for prob in predictions_tensor.flatten():
#         if prob >= threshold:
#           label = 1
#         else:
#           label =  0
#         predicted_labels.append(label)

#     predictions.append([arg_id] + predicted_labels)

# data = {'Index': [entry[0] for entry in predictions]}

# for i, label in enumerate(test_labels_df.columns[2:]):
#     data[label] = [entry[i+1] for entry in predictions]
# predictions_df = pd.DataFrame(data)


# # output_path = "/content/output/my_model_predictions.tsv"
# # predictions_df.to_csv(output_path, sep='\t', index=False)


In [ ]:
predictions = []
threshold = 0.51

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # Check for GPU or use CPU
model.to(device)  # Move model to device

for idx, row in test_df.iterrows():
    arg_id = row['Index']
    text = row['Prompt']

    # Tokenize and move inputs to the same device
    encoding = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    encoding = {key: value.to(device) for key, value in encoding.items()}

    with torch.no_grad():
        outputs = model(**encoding)

    sigmoid = torch.nn.Sigmoid()
    predictions_tensor = sigmoid(outputs.logits)  # Apply sigmoid for probabilities

    # Convert predictions to binary labels based on threshold
    predicted_labels = (predictions_tensor >= threshold).int().squeeze().tolist()  # Use tensor comparison

    predictions.append([arg_id] + predicted_labels)

# Prepare final DataFrame
data = {'Index': [entry[0] for entry in predictions]}

for i, label in enumerate(test_labels_df.columns[2:]):
    data[label] = [entry[i + 1] for entry in predictions]

predictions_df = pd.DataFrame(data)

# Uncomment to save the predictions to a file
# output_path = "/content/output/my_model_predictions.tsv"
# predictions_df.to_csv(output_path, sep='\t', index=False)


In [ ]:
predictions_df

In [ ]:
true_pred = test_labels_df[['Index', 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
true_pred

In [ ]:
true_labels = true_pred[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
pred_labels = predictions_df[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]

f1_average = f1_score(true_labels, pred_labels, average='macro')
print(f"Average F1 Score: {f1_average}")

In [ ]:

from sklearn.metrics import accuracy_score

true_labels = true_pred[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]
pred_labels = predictions_df[[ 'Antagonist', 'Protagonist',
       'Innocent', 'Bigot', 'Conspirator', 'Corrupt', 'Deceiver', 'Exploited',
       'Foreign Adversary', 'Forgotten', 'Guardian', 'Incompetent',
       'Instigator', 'Martyr', 'Peacemaker', 'Rebel', 'Saboteur', 'Scapegoat',
       'Spy', 'Terrorist', 'Traitor', 'Tyrant', 'Underdog', 'Victim',
       'Virtuous']]

exact_match_count = 0
total_count = len(true_labels)

for i in range(total_count):
  if true_labels.iloc[i].equals(pred_labels.iloc[i]):
    exact_match_count += 1

exact_match_ratio = exact_match_count / total_count

print(f"Exact Match Ratio: {exact_match_ratio}")